<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Session: Using a Database to Find Solutions for Business Problems 

#### <i>A Microsoft Presentation from the SQL Server team</i>

## Querying the Database

<p style="border-bottom: 1px solid lightgrey;"></p>


<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true">Review the Solutions Database Structure</h2>

You can build your queries by studying the ERD for this solution. We're using a very simple layout for this database, involving only three primary tables:

- **Problem**: The list of problems in the form of "We want..."
- **Solution**: Solutions composed of various components, processes and optionally services
- **AzureService**: The Azure Services (components) and optionally any services (consulting/staff aug) needed for the solution

There are also two secondary tables to hold information about the solutions and services: 

- **SolutionReferences**: Reference documents, tutorials, and cost estimates for the Solution
- **ServiceReferences**: Reference documents, tutorials, and code samples for the Services


And of course tertiary tables to handle the many-to-many joins:

- **ProblemToSolution**
- **SoltuionToService**

Here is the Entity Relationship Diagram (ERD) for the database:

<p><img style="float: left; margin: 0px 35px 35px 0px;" src="https://github.com/BuckWoody/presentations/blob/master/problemtosolution/graphics/erd.png?raw=true"></p>

You can also edit this simple design to include more elements, such as more detail on the problem, or any other specific information you want.

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Enter a Problem, A Solution, and Tie Them Together</b></p>

With the database structure in place, you can start to enter a few problems and their associated solutions. Start with ensuring that your problem or solution isn't already in the database. If it is, simply add the missing information with standard `UPDATE` statements. If you're missing both the problem *and* the solution, you can add both at the same time. 

The following code cells shows examples:


In [1]:
-- USE solutions
-- GO

/* Start by listing the problems and solutions you already have */
SELECT * FROM Problem
GO
SELECT * FROM Solution
GO


(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.112

ProblemID,ProblemName,ProblemDescription
1,Copy Data Automatically,"We want to copy certain data elements to an external location for use by others, securely."
2,Database Backups,We want to store our database backups in another location with full redundancy.
3,Offiste High Availability,"We want additional safety on our data systems, with the ability to use them globally or as a secondary read service."
4,SQL Server 2008 EOS,"SQL Server 2008 has reached end of service, and we don't want to pay for extended support. We do have applications that required SQL Server 2008 as the back-end."
5,Lift and Shift,We have a SQL Server installation on premises and we want to duplicate or move it to Azure
6,Big Data Analytics,We have a lot of data that we need to analyze in a single environment. We'd also like to use Spark.
7,Data Hub,"We have Oracle, Terradata, HDFS and other data sources we want to query using one system, without having to install and manage lots of connectors or learn to use multiple query languages."
8,Performance Improvements,"We have the need to improve our current performance, but we can't always alter the code."
9,Security Improvements,"We need to be able to use secure security enclaves, and have encryption end-to-end within the server without decoding at the client."
10,SOA Architecture,We want to create a Service Oriented Architecture and use our SQL Server Data.


SolutionID,SolutionName,SolutionDescription
1,SQL Replication to Azure VM,Use SQL Server Replication to copy data to an Azure Virtual Machine.
2,Backup to Azure,"Backup using standard T-SQL statements to Azure Storage. Has the ability to be triple-redundancy, and have global copies, or keep in a single Region. Or Backup using Azure Backup."
3,Azure Site Recovery,NULL
4,Availability Group Replica,Synchronize data between SQL on Windows to Azure Managed Instance through a vnet. Allows clusterless async replica between Linux and Azure MI.
5,SQL Server 2019 Big Data Clusters,Allows large-scale MPP data and Spark processing.
6,Storage appliance,A full Hiearchical Storage Management (HMS) solution. Provides multiple automatic benefits.
7,Copy data using a gateway,"The Azure Data Factory has the ability to establish a gateway with SQL Server and move data to any number of ""sinks"", or destinations."
8,SQL Server 2019,"SQL Server 2019 has the ability to run on Windows, linux, in Containers and Kubernetes, and can act as a Data Hub using PolyBase. It also contains numerous security and performance enhancements, some with no code-change."
9,SQL Server Virtual Machines,"SQL Server VM's are a full Instance of SQL Server running in Azure. You can bring your own license or rent one, and there are multiple sizes."
10,Azure Service Bus,"Use the SQL Server Service Broker, along with the Azure Service Bus to create a full re-active messaging system"


In [1]:
/* Enter both the problem and solution at the same time
Note: This would make a great stored procedure as well. I'll leave that to you. */

DECLARE 
  @ProblemName nvarchar(150)               = '<Enter a Short Problem Name>'
, @ProblemDescription nvarchar(max)        = '<Enter The Full Problem Description Here>'
, @SolutionName nvarchar(150)              = '<Enter a Short Solution Name>'
, @SolutionDescription nvarchar(max)       = '<Enter the Full Solution Description Here>'
, @StrengthOfSolutionToProblem nvarchar(50) = '<High, Medium, or Low. How well does this solution really answer the problem?>';

BEGIN 
	INSERT INTO [dbo].[Problem] ([ProblemName],[ProblemDescription])
	VALUES (@ProblemName, @ProblemDescription)

	/* 
	Now the Solution. Note: One Problem may have many solutions, and visa-versa. 
	If that's the case, only enter the data for the Problem or Solution, and then 
	Use the "Tie Them Together" section to select for the proper entry.
	*/

	INSERT INTO [dbo].[Solution] ([SolutionName],[SolutionDescription])
	VALUES (@SolutionName, @SolutionDescription)

	/*
	Tie them together. Uses the names as a tie-out.
	*/

	INSERT INTO [dbo].[ProblemToSolution] ([ProblemID],[SolutionID],[Strength])
	VALUES
	((SELECT ProblemID from Problem WHERE ProblemName = @ProblemName)
	,(SELECT SolutionID FROM Solution WHERE SolutionName = @SolutionName)
	, @StrengthOfSolutionToProblem)

END;

SELECT p.ProblemName, s.SolutionName
FROM Solution s
INNER JOIN ProblemToSolution p2s ON s.SolutionID = p2s.SolutionID
INNER JOIN Problem p ON p2s.ProblemID = p.ProblemID
WHERE p.ProblemName = @ProblemName;
GO

Commands completed successfully.

Total execution time: 00:00:00.042

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Add Additional Information </b></p>


With the database diagram shown above, you can now create your own scripts to add additional information to each section. A blank Code Cell is included for you below. 

- Enter any code you wish for inserts, updates and deletes. 
- Stretch assignment: Create Stored Procedures for the Inserts, Updates and Deletes. Use these in a front-end system for data management.

In [0]:
/* Your code for Inserts, Updates and Deletes goes below */

In [0]:
/* Your code for Stored Procedures goes below */

In [0]:
/* Your code Views goes below */

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

- [If you need to learn more about T-SQL queries, there's a good series here](https://channel9.msdn.com/Series/Querying-SQL-Server-2012)

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Extending the Database, using a Query Interface</p>

You can change this database to whatever you like, and if you want to set up a front-end program to view the data, there's a Power BI file included in this directory you can use as an example to get started. Note that you'll need to change the connection information for the Power BI file to your database.